In [2]:
from ipywidgets import *
import plotly.graph_objects as go
import numpy as np
import scipy.fft as fft 
import scipy.signal as signal
import matplotlib.pyplot as plt
##########################################
A = lambda t : np.sin(t/np.pi)
w_0  = lambda t : 5.0*t + 3
phi_0  = 1000
T = 10
N = 1000
###########################################


checkbox = widgets.Checkbox(
    value=False,
    description='set seed',
    disabled=False,
    indent=False,
    layout=Layout(width='100px',margin='0 0 0 30px')
)

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=1,
    step=1,
)


seed_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='seed:',
    layout=Layout(width='300px')
)

fig1 = go.Figure( layout=go.Layout(
       font=dict(family="Courier New, monospace",size=18,color="#7f7f7f"),width=580, height=580,margin=dict(l=0, r=0, t=0, b=0) ,
    legend=dict(x=0, y=1)))

fig2 = go.Figure( layout=go.Layout(
       font=dict(family="Courier New, monospace",size=18,color="#7f7f7f"),width=580, height=580,margin=dict(l=0, r=0, t=0, b=0) ,
    legend=dict(x=0, y=1)))


In [9]:
x=0
x_fft=0
z=0
PHI=0
PHI_U=0

def BFG1():
    global x,ts
    ts = np.linspace(0, T, N)
    x = A(ts) * np.cos( w_0(ts) * ts + phi_0)
    return[[ts,x]]    
    
def BFG2():
    global x,x_fft
    x_fft = fft.fft(x)
    x_fft[int(N/2):] = np.zeros(int(N/2))
    x_fft[:int(N/2)] = x_fft[:int(N/2)] * 2
    
    XX = []
    for i in range(0,x_fft.size):
        XX.append(i)
    return [[XX,np.abs(x_fft)]]

def BFG3():
    global x_fft,z
    z = fft.ifft(x_fft)
    XX=[]
    for i in range(0,z.size):
        XX.append(i)
        
    return [[XX,np.real(z)]]


def BFG4():
    global z,ts
    A_done = np.abs(z)
    XX=[]
    for i in range(0,A_done.size):
        XX.append(i)
    return [[XX,A_done]]


def BFG5():
    global z,PHI,PHI_U
    PHI = np.angle(z)
    PHI_U = np.unwrap(PHI)
    XX = []
    for i in range(0,len(PHI)):
        XX.append(i)
    return[[XX,PHI]]

def BFG6():
    global PHI_U
    phi = 1.0 / 2.0 / np.pi * np.gradient(PHI_U)

    XX=[]
    for i in range(0,phi.size):
            XX.append(i)
    return [[XX,phi]]





def css_widget(text, font_size=10):
    html = "<h1 style='font-size:"+str(font_size)+"px; margin-left: 50px; margin-top: 0;margin-bottom:0;width:350px'>" + str(text) + "</h1>"
    W = widgets.HTML(value=html)
    return W

In [10]:

step_btn = widgets.Button(description='refresh', button_style='primary')

def custom_layout(w,h):

    ll=Layout(width=str(w) + 'px', height=str(h) + 'px', border='solid')

    return ll 


def showText(text):
    print("\t", text)





def function(widget, figure, fun_for_data):
    
    figure.data = []       
    data = fun_for_data() 
 
    for values in data:
        figure.add_trace(go.Scatter(x=values[0], y=values[1]))
        
        

        


    figure.show()


def custom_display():
    
    text=""" На цифровой сигнал подается БГШ (белый гаусс. шум). Пропускаем через фильтр Баттерворта. Рассчитать отношения мощностей до и после фильтрации.
    В работе используются два фильтра: фильтр Баттерворта и встроенный в пакет scipy.signal lowess """
    
    
    ui = HBox([step_btn, checkbox, seed_slider], layout=Layout(margin='0 0 0 33px'))
    HH_ui1 = VBox([ui,interactive_plot1])
    HH_ui2 = VBox([ui,interactive_plot2])
    HH_ui3 = VBox([ui,interactive_plot3])
    HH_ui4 = VBox([ui,interactive_plot4])
    HH_ui5 = VBox([ui,interactive_plot5])
    HH_ui6 = VBox([ui,interactive_plot6])
    
    
    tab = widgets.Tab(children = [HH_ui1, HH_ui2,HH_ui3,HH_ui4,HH_ui5,HH_ui6 ])
    tab.set_title(0, '1')
    tab.set_title(1, '2')
    tab.set_title(2, '3')
    tab.set_title(3, '4')
    tab.set_title(4, '5')
    tab.set_title(5, '6')
    
    
    text_for_slider1= "ОТНОШЕНИЕ ЧИСТОГО К БАТТЕР"
    text_for_slider2= "ОТНОШЕНИЕ ЧИСТОГО К LOWESS"
    text_for_slider3= "ОТНОШЕНИЕ ЧИСТОГО К ШУМУ"

    
    

    
    HHH_2 = (Box([tab], layout = custom_layout(700,700)))
    
    
    
    Htext = VBox([css_widget("Задача 1",15),])
    
    
    HHH_3 = (Box([Htext], layout = custom_layout(500,700)))
    
    item_2 = HBox([HHH_2,HHH_3])

    display(VBox([item_2]))
    
    toggle_visibility(None)


def toggle_visibility(button):
    seed_slider.layout.visibility = 'hidden' if (checkbox.value == False) else  'visible'


def clicker(b):
    global SEED
    SEED = r.randrange(1, 100)
    slider.value = 1 if (slider.value == 0) else  0  
    custom_display()
       
step_btn.on_click(clicker)
checkbox.observe(toggle_visibility)

interactive_plot1 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG1) })
interactive_plot2 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG2) })
interactive_plot3 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG3) })
interactive_plot4 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG4) })
interactive_plot5 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG5) })
interactive_plot6 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG6) })

custom_display()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))